In [1]:
from astropy.io import fits
from astropy.table import Table
import random, os
import numpy as np

In [24]:
# with fits.open('gll_psc_v16.fit') as hdu:
#     FGL3 = Table(hdu[1].data)
#     headers = hdu[0].header
#     cols = hdu[1].columns
#     data_3FGL = hdu[1].data
 
# cols.info()
# print(FGL3)
# # print(data_3FGL[0][1])
# # print(data_3FGL['Source_Name'])
# # print(data_3FGL[0])
# #print(data_3FGL[2]['CLASS1'])
# pulsars = []
# agns = []
# others = []
# unclass = []
# for i in range(len(data_3FGL)): #need to find out which classes count for AGNs and pulsars
#     #pulsars (classification labels in 3FGL: PSR, psr)
#     if data_3FGL[i]['CLASS1'] == 'PSR' or data_3FGL[i]['CLASS1'] == 'psr':
#         pulsars.append(data_3FGL[i])
#     #AGNs (classification labels in the 3FGL catalog: agn, FSRQ, fsrq, BLL, bll, BCU, bcu, RDG, rdg, NLSY1, nlsy1, ssrq, and sey)
#     elif data_3FGL[i]['CLASS1'] == 'agn' or data_3FGL[i]['CLASS1'] == 'FSRQ' or data_3FGL[i]['CLASS1'] == 'fsrq' or data_3FGL[i]['CLASS1'] == 'BLL' or data_3FGL[i]['CLASS1'] == 'bll' or data_3FGL[i]['CLASS1'] == 'BCU' or data_3FGL[i]['CLASS1'] == 'bcu' or data_3FGL[i]['CLASS1'] == 'RDG' or data_3FGL[i]['CLASS1'] == 'rdg' or data_3FGL[i]['CLASS1'] == 'NLSY1' or data_3FGL[i]['CLASS1'] == 'nlsy1' or data_3FGL[i]['CLASS1'] == 'ssrq' or data_3FGL[i]['CLASS1'] == 'sey':
#         agns.append(data_3FGL[i])
#     elif data_3FGL[i]['CLASS1'] == '':
#         others.append(data_3FGL[i])
#     else:
#         unclass.append(data_3FGL[i])


In [2]:
hdul_3fgl = fits.open('gll_psc_v16.fit') #add file directory data/

classes = hdul_3fgl[1].data['CLASS1']
agn_classes = ['psr  ', 'agn  ', 'FSRQ ', 'fsrq ', 'BLL  ', 'bll  ', 'BCU  ', 'bcu  ', 'RDG  ', 'rdg  ', 'NLSY1', 'nlsy1', 'ssrq ', 'sey  ']
pulsar_classes = ['PSR  ', 'psr  ']
no_class = '     '
agn_mask = np.isin(classes, agn_classes)
pulsar_mask = np.isin(classes, pulsar_classes)
noclass_mask = classes == no_class
bad_data_mask = hdul_3fgl[1].data['Signif_Curve'] == 0.0 # Causes ln(0.0)

In [53]:
oversample = sum(agn_mask) - sum(pulsar_mask)

duplicated = []
for i in range(classes.size):
    duplicated.append(hdul_3fgl[1].data[i])
    
# pulsars = []
# for i in range(classes.size):
#     #pulsars (classification labels in 3FGL: PSR, psr)
#     if hdul_3fgl[1].data[i]['CLASS1'] == 'PSR' or data_3FGL[i]['CLASS1'] == 'psr':
#         pulsars.append(data_3FGL[i])

pulsars = hdul_3fgl[1].data[pulsar_mask & ~bad_data_mask]
    
for i in range(oversample):
    duplicated.append(random.choice(pulsars))
    
# duplicated = np.array(duplicated)
    
# print(overfit)
# print(len(data_3FGL))
# print(len(pulsars))
# print(len(agns))
# print(len(over_3FGL))
print(classes.size)
print(oversample)
print(len(duplicated))

3034
1601
4635


In [3]:
oversample = sum(agn_mask) - sum(pulsar_mask)

nrows1 = hdul_3fgl[1].data.shape[0]
# nrows1 = data.shape[0]
nrows = nrows1 + oversample
hdu = fits.BinTableHDU.from_columns(hdul_3fgl[1].columns, nrows=nrows)

pulsars = hdul_3fgl[1].data[pulsar_mask & ~bad_data_mask]

for colname in hdul_3fgl[1].columns.names:
    hdu.data[colname][nrows1:] = random.choice(pulsars[colname])


# try:
#     hdul_v2.close()
#     os.remove('temptable.fits')
# except:
#     print()
#hdul_v2.close()
hdu.writeto('temptable.fits')
hdul_v2 = fits.open('temptable.fits')

classes = hdul_v2[1].data['CLASS1']
agn_mask = np.isin(classes, agn_classes)
pulsar_mask = np.isin(classes, pulsar_classes)
noclass_mask = classes == no_class
bad_data_mask = hdul_v2[1].data['Signif_Curve'] == 0.0

data = hdul_v2[1].data[(agn_mask | pulsar_mask) & ~bad_data_mask] # data is both pulsars and AGNs
pulsars = hdul_v2[1].data[pulsar_mask & ~bad_data_mask]
agns = hdul_v2[1].data[agn_mask & ~bad_data_mask]

# for i in range(oversample):
#     data.add_row(np.random.choice(pulsars))



# print(classes.size)
# print(oversample)

5


In [4]:
# The easy ones
glat = data['GLAT']
glon = data['GLON']
ln_energy_flux100 = np.log(data['Energy_Flux100'])
ln_unc_energy_flux100 = np.log(data['Unc_Energy_Flux100'])
ln_signif_curve = np.log(data['Signif_Curve'])
ln_var_index = np.log(data['Variability_Index'])

# Hardness ratios
ef1 = data['Flux100_300']
ef2 = data['Flux300_1000']
ef3 = data['Flux1000_3000']
ef4 = data['Flux3000_10000']
ef5 = data['Flux10000_100000']
hr12 = (ef2 - ef1) / (ef2 + ef1)
hr23 = (ef3 - ef2) / (ef3 + ef2)
hr34 = (ef4 - ef3) / (ef4 + ef3)
hr45 = (ef5 - ef4) / (ef5 + ef4)

# 500 MeV index
alpha = data['Spectral_Index']
beta = data['beta']
gamma = data['Spectral_Index']
b = data['Exp_Index']
E_c = data['Cutoff'] # In MeV
E_0 = data['Pivot_Energy'] # In MeV
mev_500_index = np.zeros(data.shape)
for i, point in enumerate(data):
    if point['SpectrumType'] in ['PowerLaw', 'PLExpCutoff', 'PLSuperExpCutoff']:
        if b[i] == float('-inf'):
            b[i] = 1
        mev_500_index[i] = gamma[i] + b[i] * (500 / E_c[i])**b[i]
    else:
        mev_500_index[i] = alpha[i] + 2*beta[i] * np.log(500 / E_0[i])


In [5]:
in_data = np.vstack((glat, glon, ln_energy_flux100, ln_unc_energy_flux100, ln_signif_curve, ln_var_index, hr12, hr23, hr34, hr45, mev_500_index))
out_data = np.isin(data['CLASS1'], agn_classes).astype(float)
np.savez_compressed('3fgl_over_data.npz', in_data=in_data, out_data=out_data)

hdul_v2.close()
os.remove('temptable.fits')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'temptable.fits'